In [1]:
import os
import json
import openai
from langchain_community.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('suksoDF.csv')

# 데이터 프레임 확인
df.head(3)

,name,address,overview,imglink,generalInfo,roomInfo
0,가경재,경상북도 안동시 하회남촌길 69-5,"가경재는 안동 하회마을 남쪽에 자리한 초가 한옥이다. 객실은 총 5개로, 안채는 중...",['http://tong.visitkorea.or.kr/cms/resource/00...,"-숙소 기본정보-\n객실유형: 한실\n객실 수: 2\n환불 규정: 7일전 100%,...",['\n**- 객실정보 1 -**객실명: 사랑채 아랫방\n객실크기: 0\n객실수: ...
1,가락관광호텔,서울특별시 송파구 송파대로28길 5,"가락 관광호텔은 동양 최대의 가락동농수산물시장과 인접해 있고 주변에 잠실 롯데월드,...",[],"-숙소 기본정보-\n객실유형: 스탠다드 더블/트윈, 디럭스 더블, 주니어 스위트, ...",['\n**- 객실정보 1 -**객실명: 디럭스 더블\n객실크기: 10\n객실수: ...
2,가락청,전북특별자치도 전주시 완산구 한지길 68,더할 가 (加) + 즐거울 락 (樂) 여행의 즐거움을 더한다는 뜻을 담은 한옥 숙박...,[],-숙소 기본정보-\n객실 수: 5\n체크인: 15:00\n체크아웃: 10:00\n조...,['\n**- 객실정보 1 -**객실명: 아쟁\n객실크기: 0\n객실수: 0\n기준...


In [2]:
# 2. 텍스트 벡터화: OpenAI 임베딩 모델 사용
embeddings = OpenAIEmbeddings( #← OpenAIEmbedings를 초기화
    model="text-embedding-ada-002"
)

/home/encore/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [3]:
from langchain.vectorstores import Chroma
db = Chroma(
    persist_directory='./sukso_data',
    embedding_function=embeddings
)

# 4. 질의-응답 시스템 생성: ChromaDB에서 유사한 문서를 검색하고 OpenAI 모델로 답변 생성
llm = ChatOpenAI(model="gpt-3.5-turbo")

# 5. RetrievalQA 객체를 사용하여 검색 및 답변 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="map_reduce", 
    retriever=db.as_retriever()
)

/home/encore/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:
from langchain.schema import Document
documents = [
    Document(page_content=f"관광지명: {row['name']} 주소: {row['address']},  소개: {row['overview']}, 숙소정보: {row['generalInfo']}, 객실정보: {row['roomInfo']} ", metadata={"id": idx})
    for idx, row in df.iterrows()
]

# 문서 배치를 나누는 함수
def batch_documents(documents, batch_size):
    for i in range(0, len(documents), batch_size):
        yield documents[i:i + batch_size]
        
# 배치 크기 설정
batch_size = 50

# 배치로 나누어 처리
for batch in batch_documents(documents, batch_size):
    try:
        db.add_documents(batch)
        db.persist()
        print(f"Batch of size {batch_size} added successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")

/home/encore/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
Batch of size 50 added successfully.
B

In [7]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models import ChatOpenAI 
from langchain.schema import HumanMessage
chat = ChatOpenAI(
    model = 'gpt-4o-mini-2024-07-18'
)

def get_answer_from_db(user_query):
    #  ChromaDB에서 유사한 문서 검색
    results = db.similarity_search(user_query, k=3)  # 상위 3개 유사도 검색

    #  검색된 결과를 하나의 컨텍스트로 묶기
    context = "\n".join([result.page_content for result in results])

    #  ChatGPT에 전달할 메시지 구성
    messages = [
        SystemMessage(content="당신은 숙소 도우미 챗봇입니다."),
        HumanMessage(content=f"사용자 질문: {user_query}\n\n참고할 정보:\n{context}\n\n이 정보를 기반으로 정확하게 답변해 주세요.")
    ]

    #  ChatGPT로부터 답변 받기
    response = chat(messages)

    return response.content

In [8]:
query = "가락청은 조리 가능한 숙소야?"
answer = get_answer_from_db(query)
print(answer)

/home/encore/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


가락청은 조리 가능한 숙소가 아닙니다. 숙소 정보에 따르면 가락청은 조리 가능한 옵션이 제공되지 않는 한옥 고택입니다. 다른 문의가 있으시면 언제든지 말씀해 주세요!


In [12]:
query = "가보호텔 숙소 주소 알려줘"
answer = get_answer_from_db(query)
print(answer)

가보호텔의 주소는 **경기도 평택시 평택5로76번길 18-10**입니다. 추가적인 문의사항이 있으시면 031-658-7700으로 연락하시면 됩니다.


In [11]:
query = "강원도 숙소 추천해줘"
answer = get_answer_from_db(query)
print(answer)

강원도에서 추천할 만한 숙소를 몇 곳 소개해드릴게요!

1. **흙과소나무**
   - **주소**: 강원특별자치도 정선군 남면 자뭇골길 137-12
   - **소개**: 강원도의 시원한 산세를 감상할 수 있는 곳으로, 깨끗한 공기 덕에 휴양을 즐길 수 있습니다. 목구조의 아늑한 공간과 바비큐 시설이 마련되어 있어 친구, 연인, 가족과 함께 즐기기 좋습니다.
   - **객실 수**: 5실 (최대 수용인원: 28명)
   - **체크인/체크아웃**: 15:00 / 비수기 12:00, 성수기 11:00
   - **문의**: 033-591-1159

2. **산골초가펜션**
   - **주소**: 강원특별자치도 영월군 북면 굴앞마을길 64-152
   - **소개**: 시골 할머니 집처럼 아늑한 분위기를 느낄 수 있는 숙소입니다. 자연과 마주하며 다양한 액티비티를 즐길 수 있습니다.
   - **객실 수**: 9실 (독채형)
   - **체크인/체크아웃**: 14:00 / 12:00
   - **문의**: 010-6299-0395

3. **한우리펜션**
   - **주소**: 강원특별자치도 영월군 무릉도원면 술면길 6-5
   - **소개**: 상쾌한 자연 속에서 조용한 휴식을 취할 수 있는 곳입니다. 바로 근처에 계곡도 있어 여름철 물놀이에도 좋습니다.
   - **객실 수**: 6실
   - **체크인/체크아웃**: 14:00 / 12:00
   - **문의**: 0507-1320-5041

각 숙소는 독특한 매력을 가지고 있으니, 원하는 스타일과 조건에 맞춰 선택하시면 좋을 것 같습니다! 추가적인 궁금한 점이 있으시면 언제든지 문의해주세요.


In [14]:
query = "프라임리조트 숙소의 주중성수기 가격 알려줘"
answer = get_answer_from_db(query)
print(answer)

프라임 리조트의 주중 성수기 가격은 다음과 같습니다:

- 15평형 듀플렉스: 150,000원
- 22평형 듀플렉스: 220,000원
- 25평형 듀플렉스: 250,000원
- 35평형 듀플렉스: 350,000원
- 40평형 별장형: 400,000원
- 60평형 별장: 600,000원

각 객실에 따라 가격이 다르니, 원하는 객실의 가격을 확인하시기 바랍니다. 추가로 궁금한 사항이 있으시면 언제든지 문의해 주세요!


In [15]:
query = "포천 아이러브펜션 숙소의 객실 정보 알려줘"
answer = get_answer_from_db(query)
print(answer)

포천 아이러브펜션의 객실 정보는 다음과 같습니다:

### 기본 정보
- **주소:** 경기도 포천시 영북면 산정호수로411번길 112-22
- **객실 유형:** 원룸형 침대/온돌, 복층형 온돌, 소나무관, 대나무관, 별채
- **객실 수:** 13실
- **체크인:** 15:00
- **체크아웃:** 익일 11:00
- **조리 가능 여부:** 가능
- **전체 수용 인원:** 109명
- **문의 전화:** 031-532-7710

### 객실 정보

1. **백합/매화/동백/철쭉**
   - **객실 크기:** 7평
   - **객실 수:** 4
   - **기준 인원:** 2
   - **비수기 주중 최소:** 60,000원
   - **비수기 주말 최소:** 100,000원
   - **주중 성수기:** 110,000원
   - **주말 성수기:** 110,000원
   - **시설:** 목욕시설, 에어컨, 냉장고, 드라이기, 테이블

2. **대나무관**
   - **객실 크기:** 22평
   - **객실 수:** 1
   - **기준 인원:** 10
   - **비수기 주중 최소:** 150,000원
   - **비수기 주말 최소:** 250,000원
   - **주중 성수기:** 250,000원
   - **주말 성수기:** 250,000원
   - **시설:** 목욕시설, 에어컨, TV, 냉장고, 테이블

3. **소나무관**
   - **객실 크기:** 18평
   - **객실 수:** 1
   - **기준 인원:** 10
   - **비수기 주중 최소:** 130,000원
   - **비수기 주말 최소:** 220,000원
   - **주중 성수기:** 220,000원
   - **주말 성수기:** 220,000원
   - **시설:** 목욕시설, 에어컨, TV, 냉장고, 테이블

4. **별채**
   - **객실 크기:** 6평
   - **객실 수:** 1
   - **기준 인원:** 2
   - **비수기 주중 최소:** 50,0

In [19]:
query = "강구안 게스트하우스 숙소의 총 객실 수 알려줘"
answer = get_answer_from_db(query)
print(answer)

강구안 게스트하우스의 총 객실 수는 6실입니다.


In [21]:
query = "팔공 에밀리아호텔의 편의시설을 알려줘"
answer = get_answer_from_db(query)
print(answer)

팔공 에밀리아호텔은 다음과 같은 편의시설을 제공합니다:

- **객실 유형**: 양실과 한실을 포함하여 총 38실의 객실이 있습니다.
- **체크인/체크아웃**: 체크인 시간은 14:00, 체크아웃 시간은 12:00입니다.
- **조리 가능 여부**: 객실 내 조리는 불가합니다.
- **주차**: 주차가 가능합니다.
- **픽업 서비스**: 픽업 서비스가 제공됩니다.
- **편의시설**:
  - 다양한 크기의 연회장
  - 갤러리
  - 카페
  - 레스토랑
  - 펍
  - 노래방

이 외에도 팔공산의 자연경관 속에서 평화로운 휴식을 제공하며, 세미나 및 다양한 행사에 적합한 공간을 갖추고 있습니다.

예약이나 문의는 전화(0507-1406-1000) 또는 [이 웹사이트](http://www.emillia.co.kr/)를 통해 가능합니다.
